# Generating Report Card From Excel

In [4]:
# importing the required libraries

import pandas as pd 
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.platypus import Table, TableStyle
from reportlab.lib import colors
import os

In [5]:
# uploading the file
file_path = "student_scores.xlsx"

In [9]:
def generate_report_card(file_path):
    try:

        data = pd.read_excel(file_path)   #reading the file
        required_columns = {"Student ID", "Name", "Subject", "Score"}  #validate columns
        if not required_columns.issubset(data.columns):
            raise ValueError(f"Excel file must contain the columns: {required_columns}")
            
        if data.isnull().any().any():  #Handling missing or invalide data
            print("Warning: Missing data found. Filling with default values.")
            data.fillna({"Score": 0}, inplace=True)
        
        grouped = data.groupby(["Student ID", "Name"])   #grouping and calculating metrics
        total_scores = grouped["Score"].sum()
        avg_scores = grouped["Score"].mean()
        
        for (student_id, student_name), group in grouped:   #generating pdf for each student
            # File name
            file_name = f"report_card_{student_id}.pdf"
            
            # Creating a PDF canvas
            c = canvas.Canvas(file_name, pagesize=letter)
            c.setFont("Helvetica-Bold", 14)
            
            # Title
            c.drawString(100, 750, f"Report Card for {student_name}")
            c.setFont("Helvetica", 12)
            
            # Total and Average Scores
            c.drawString(100, 720, f"Total Score: {total_scores.loc[(student_id, student_name)]}")
            c.drawString(100, 700, f"Average Score: {avg_scores.loc[(student_id, student_name)]:.2f}")
            
            # Subject-wise Scores Table
            subject_scores = group[["Subject", "Score"]]
            data_list = [["Subject", "Score"]] + subject_scores.values.tolist()
            table = Table(data_list, colWidths=[200, 100])
            
            # Adding table styles
            style = TableStyle([
                ("BACKGROUND", (0, 0), (-1, 0), colors.grey),
                ("TEXTCOLOR", (0, 0), (-1, 0), colors.whitesmoke),
                ("ALIGN", (0, 0), (-1, -1), "CENTER"),
                ("FONTNAME", (0, 0), (-1, 0), "Helvetica-Bold"),
                ("BOTTOMPADDING", (0, 0), (-1, 0), 12),
                ("BACKGROUND", (0, 1), (-1, -1), colors.beige),
                ("GRID", (0, 0), (-1, -1), 1, colors.black),
            ])
            table.setStyle(style)
            
            # Add table to PDF
            table.wrapOn(c, 100, 600)
            table.drawOn(c, 100, 550)
            
            # Save PDF
            c.save()
            print(f"Generated: {file_name}")
    
    except FileNotFoundError:
        print("Error: The specified file does not exist.")
    except ValueError as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

In [10]:
generate_report_card(file_path)

Generated: report_card_101.pdf
Generated: report_card_102.pdf
Generated: report_card_103.pdf


## Explanation

1. Reading the Data:
Use the pandas library to load the data from an Excel file (student_scores.xlsx).
Validate that the necessary columns (Student ID, Name, Subject, Score) are present. Handle missing or invalid data by filling or discarding incomplete rows.

2. Aggregating Data:
Group the data by Student ID and Name to compute each student's total and average scores.
Extract subject-wise scores for individual students for detailed reporting.

3. Generating PDF Report Cards:
Use the ReportLab library to create PDF files.
Each report includes:
  The student’s name.
  Their total score and average score.
  A table displaying their subject-wise scores.
Apply styling to the table for better readability.

4. Saving PDF Files:
Name each file using the format report_card_<StudentID>.pdf and save it in the working directory.

5. Error Handling:
Include error handling for file-related issues (e.g., missing file), missing columns, or unexpected data formats.